In [1]:
import requests
import json

import time
from datetime import datetime, time as dt_time

import schedule

import sys
import os
import pandas as pd
import numpy as np
import decimal
from datetime import date

import requests
from pyathena import connect

import math
from math import floor, ceil

from dataclasses import dataclass
from typing import Tuple, Dict, Any
import numpy as np

import pandas_gbq
from google.cloud import bigquery

from datetime import date
import pandas as pd
from google.cloud import bigquery
from typing import List, Optional

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from datetime import datetime


list_thematics = ["DEFI11", "WEB311", "META11", "FOMO11"]
list_nasdaq = ["HASH11", "SOLH11", "BITH11", "ETHE11", "XRPH11"]
from google.cloud import bigquery
client = bigquery.Client("hdx-data-platform")


sys.path.append('G:\Drives compartilhados\Investment Management\Trading_Routines_logics')
from Post_trades_api import TradingAPIManager


<>:46: SyntaxWarning: invalid escape sequence '\D'
<>:46: SyntaxWarning: invalid escape sequence '\D'
C:\Users\lucas.afonso_hashdex\AppData\Local\Temp\ipykernel_25444\4184113974.py:46: SyntaxWarning: invalid escape sequence '\D'
  sys.path.append('G:\Drives compartilhados\Investment Management\Trading_Routines_logics')
c:\Users\lucas.afonso_hashdex\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
C:\Users\lucas.afonso_hashdex\AppData\Local\Temp\ipykernel_25444\4184113974.py:46: SyntaxWarning: invalid escape sequence '\D'
  sys.path.append('G:\Drives compartilhados\Investment Management\Trading_Routi

ModuleNotFoundError: No module named 'Post_trades_api'

In [ ]:
from google.cloud import bigquery
from typing import Optional

def truncate_primary_requests_table() -> None:
    """
    Deletes all records from the primary_requests table in BigQuery.
    
    Returns:
        None
    
    Raises:
        Exception: If the deletion operation fails
    """
    try:
        # Define BigQuery parameters
        project_id = "hdx-data-platform-users"
        table_id = "team_trading.primary_requests"
        
        # Initialize BigQuery client
        client = bigquery.Client(project=project_id)
        
        # Create delete query
        delete_query = f"""
        DELETE FROM `{project_id}.{table_id}`
        WHERE TRUE
        """
        
        # Execute query
        query_job = client.query(delete_query)
        query_job.result()  # Wait for query to complete
        
        print(f"Successfully deleted all records from {table_id}")
        
    except Exception as e:
        print(f"Error truncating table: {str(e)}")
        raise
    
    
truncate_primary_requests_table()


In [106]:
### Auxiliar functions
def format_brl(value):
    """Format number to Brazilian currency format"""
    return f"R$ {value:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")

def create_email_sentences(df):
    # Dictionary to translate operation types
    side_translation = {
        'BUY': 'compra',
        'SELL': 'venda'
    }
    
    # List to store sentences
    sentences = []
    
    # Iterate through DataFrame rows
    for _, row in df.iterrows():
        # Get values from row
        asset = row['Asset']
        side = side_translation[row['Side'].upper()]
        spread_side = "abaixo" if row['Side'] == "SELL" else "acima"
        amount = format_brl(row['Amount'])  # Format the amount in BRL
        
        # Create sentence
        sentence = f"Favor iniciar uma {side} de {asset} até {amount}, até 30 bps {spread_side} do justo"
        sentences.append(sentence)
    
    return sentences

def create_primary_sentences(df: pd.DataFrame) -> list:
    import locale
    locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')  # for Linux/macOS
    # On Windows, use: locale.setlocale(locale.LC_ALL, 'Portuguese_Brazil.1252')

    primary_values = get_todays_primary_values()
    frases = []

    for _, row in df.iterrows():
        fund_name = row['fund_name']
        cnpj = row['fund_cnpj']
        asset = row['asset']
        side = row['side']
        units = row['primary_units']

        valor_row = primary_values.loc[primary_values['ETF'] == asset]
        if not valor_row.empty:
            valor_estimado = valor_row.iloc[0]['Estimated Opening Amount']
            valor_formatado = f"R$ {valor_estimado:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
        else:
            valor_formatado = 'R$ N/A'

        if side.upper() == 'BUY':
            frase = f"For {fund_name} (CNPJ: {cnpj}): Creation of {units} units of {asset} @ {valor_formatado}"
        elif side.upper() == 'SELL':
            frase = f"For {fund_name} (CNPJ: {cnpj}): Redemption of {units} units of {asset} @ {valor_formatado}"
        else:
            frase = f"{fund_name} (CNPJ: {cnpj}) - side '{side}' not recognized."

        frases.append(frase)

    return frases

def get_todays_primary_values():
    etfs = ['HASH11', 'BITH11', 'ETHE11', 'SOLH11', 'XRPH11', 'DEFI11', 'WEB311', 'META11']
    url = "https://api2.hashdex.io/marketdata/v2/inav/"

    results = []

    for etf in etfs:
        response = requests.get(url + etf)
        data = response.json()

        asset = data['info']['fundName']
        lot_units = data['info']['numberOfSharesPerCreationUnit']
        valorestimated_open = data['info']['estimatedNetAssetValuePerShareForNextDate']
        valorInav = data['inavPerShare']
        
        opening_amount = lot_units * valorestimated_open
        inav_amount = lot_units * valorInav

        results.append({
            'ETF': etf,
            'Lot Units': lot_units,
            'Estimated Opening iNAV': valorestimated_open,
            'iNAV per Share': valorInav,
            'Estimated Opening Amount': opening_amount,
            'Current Lot Amount': inav_amount
        })

    return pd.DataFrame(results)
        
def filter_primary_values(asset_trade):
    
    primary_values = get_todays_primary_values()
    primary_values = primary_values[primary_values['ETF'] == asset_trade]
    print(primary_values)
    
    return primary_values
      
def calculate_primary_request(target_trade, lot_amount):
    
    # Calculate primary units request
    primary_units_request = ceil(target_trade / lot_amount) if (
        target_trade - (floor(target_trade / lot_amount)) * lot_amount > lot_amount / 2
    ) else floor(target_trade / lot_amount)
    
    return primary_units_request

def extract_asset_lists():
    """Extract unique onshore and offshore assets from funds configuration"""
    funds_data_static = pd.read_json(r'G:\Drives compartilhados\Investment Management\Trading_Process\Funds_stats_dict.json')
    print(funds_data_static)
    onshore_assets = set()
    offshore_assets = set()
    
    for fund in funds_data_static["funds"]:
        # Add onshore assets (handle None values)
        onshore_list = fund.get("assets_onshore")
        if onshore_list is not None:
            onshore_assets.update(onshore_list)
        
        # Add offshore assets (handle None values)
        offshore_list = fund.get("assets_offshore")
        if offshore_list is not None:
            offshore_assets.update(offshore_list)
    
    return sorted(list(onshore_assets)), sorted(list(offshore_assets))

def get_and_organize_trades():
    
    trades_data = pd.read_csv(r'G:\Drives compartilhados\Investment Management\Trading_Process\Daily_trades_use.csv')
    print(trades_data)
    simple_order, rebalance_order = trades_data[trades_data["Strategy"] == "SimpleOrder"], trades_data[trades_data["Strategy"] == "RebalanceOrder"]
    # print(simple_order)
    
    df_grouped_simple_order = simple_order.groupby(['Asset', 'Side'], as_index=False)['Amount'].sum()
    df_grouped_rebalance_order = rebalance_order.groupby(['Asset', 'Side'], as_index=False)['Amount'].sum()
    
    sentences = create_email_sentences(df_grouped_simple_order)
    print(sentences)
    on_shore_assets, off_shore_assets = extract_asset_lists()

    # Filter trades by onshore and offshore assets
    off_shore_trades = trades_data[trades_data["Asset"].isin(off_shore_assets)]
    on_shore_trades = trades_data[trades_data["Asset"].isin(on_shore_assets)]
    
    return (simple_order, rebalance_order, sentences,
            df_grouped_simple_order, df_grouped_rebalance_order,  
            trades_data, on_shore_trades, off_shore_trades)

def group_orders_for_broker(allocation_df: pd.DataFrame) -> pd.DataFrame:
    """Group allocation results by Asset and Side for broker submission"""
    # Filter for non-zero allocations only
    orders_to_send = allocation_df[allocation_df['Qty_Allocation'] > 0].copy()
    
    if orders_to_send.empty:
        return pd.DataFrame()
    
    # Group by Asset and Side (similar to your existing grouping pattern)
    grouped = orders_to_send.groupby(['Asset', 'Side']).agg({
        'Qty_Allocation': 'sum',
        'Financial_Allocation': 'sum',
        'Price_Executed': 'mean'
    }).reset_index()
    
    # Rename to match your existing pattern
    grouped.columns = ['Asset', 'Side', 'Amount', 'Financial_Total', 'Avg_Price']
    
    return grouped

def create_adjustment_email_sentences(allocation_df: pd.DataFrame, executed_trades_df: pd.DataFrame):
    """Create comprehensive email with tables showing targets vs executed trades"""
    
    if allocation_df.empty:
        return "No adjustment orders needed."
    
    # Get all possible ETFs from your lists
    all_etfs = list_nasdaq + list_thematics  # Combine your ETF lists
    
    # Create summary for buy and sell sides
    buy_summary = []
    sell_summary = []
    
    for etf in all_etfs:
        for side in ['BUY', 'SELL']:
            # Calculate target from what SHOULD be executed (use Target_to_Adjustment directly)
            etf_allocation = allocation_df[
                (allocation_df['Asset'] == etf) & (allocation_df['Side'] == side)
            ]
            
            # Sum the Target_to_Adjustment to get what still needs to be done
            if not etf_allocation.empty:
                adjustment_needed = etf_allocation['Target_to_Adjustment'].sum()
                total_target = etf_allocation['Target_Value'].sum()
            else:
                adjustment_needed = 0.0
                total_target = 0.0
            
            # Calculate executed (from executed trades) 
            side_code = 'C' if side == 'BUY' else 'V'
            etf_executed = executed_trades_df[
                (executed_trades_df['ATIVO'] == etf) & (executed_trades_df['SIDE'] == side_code)
            ]
            
            if not etf_executed.empty:
                executed_financial = etf_executed['Financeiro'].sum()
            else:
                executed_financial = 0.0
            
            # The target should be executed + adjustment_needed
            target_financial = executed_financial + adjustment_needed
            
            # Format values
            target_formatted = format_brl(target_financial)
            executed_formatted = format_brl(executed_financial)
            diff_formatted = format_brl(adjustment_needed)
            
            # Add to appropriate summary
            row_data = {
                'ETF': etf,
                'Financeiro_target': target_formatted,
                'Financeiro_exec': executed_formatted,
                'Diff': diff_formatted
            }
            
            if side == 'BUY':
                buy_summary.append(row_data)
            else:
                sell_summary.append(row_data)
    
    # Create email body
    email_body = "Trading Adjustment Instructions\n\n"
    
    # Add observations
    email_body += "Obs: Favor verificar na tabela abaixo o que já foi executado e usar a coluna 'Diff target - exec' como parâmetro para as ordens\n"
    email_body += "Obs2: Favor não entrar com essas ordens no leilão, paramos no final do mercado\n\n"
    
    # Create buy table
    email_body += "ETF <compra>\tFinanceiro target\tFinanceiro exec.\tDiff target - exec.\n"
    for row in buy_summary:
        email_body += f"{row['ETF']}\t{row['Financeiro_target']}\t{row['Financeiro_exec']}\t{row['Diff']}\n"
    
    email_body += "\n"
    
    # Create sell table
    email_body += "ETF <venda>\tFinanceiro target\tFinanceiro exec.\tDiff target - exec.\n"
    for row in sell_summary:
        email_body += f"{row['ETF']}\t{row['Financeiro_target']}\t{row['Financeiro_exec']}\t{row['Diff']}\n"
    
    # Add specific order instructions for non-zero differences
    email_body += "\n=== ORDENS ESPECÍFICAS ===\n"
    
    # Create specific order sentences for non-zero differences
    order_sentences = []
    
    # Check buy orders
    for row in buy_summary:
        if "R$ 0,00" not in row['Diff'] and row['Diff'] != "R$ 0,00":
            # Extract numeric value for comparison
            diff_value = float(row['Diff'].replace('R$ ', '').replace('.', '').replace(',', '.'))
            if abs(diff_value) > 100:  # Only significant amounts
                order_sentences.append(f"Favor iniciar uma compra de {row['ETF']} até {row['Diff']}, até 30 bps acima do justo (Adjustment)")
    
    # Check sell orders  
    for row in sell_summary:
        if "R$ 0,00" not in row['Diff'] and row['Diff'] != "R$ 0,00":
            # Extract numeric value for comparison
            diff_value = float(row['Diff'].replace('R$ ', '').replace('.', '').replace(',', '.'))
            if abs(diff_value) > 100:  # Only significant amounts
                order_sentences.append(f"Favor iniciar uma venda de {row['ETF']} até {row['Diff']}, até 30 bps abaixo do justo (Adjustment)")
    
    if order_sentences:
        for i, sentence in enumerate(order_sentences, 1):
            email_body += f"{i}. {sentence}\n"
    else:
        email_body += "Nenhuma ordem de ajuste necessária no momento.\n"
    
    return email_body

def create_final_allocation_for_broker(allocation_df: pd.DataFrame) -> pd.DataFrame:
    """
    Create final consolidated allocation for broker (secondary + adjustment orders combined)
    
    Args:
        allocation_df: DataFrame from calculate_update_secondary_values()
        
    Returns:
        DataFrame with total quantities to execute per fund/asset/side
    """
    
    # Filter for all orders with quantities > 0 (both MainOrder and AdjustmentOrder)
    all_orders = allocation_df[allocation_df['Qty_Allocation'] > 0].copy()
    
    if all_orders.empty:
        print("No orders to allocate")
        return pd.DataFrame(columns=['trade_date', 'asset', 'side', 'asset_amount', 'exchange_code', 'fund', 'cnpj'])
    
    # Group by Fund + Asset + Side + CNPJ and sum quantities
    grouped = all_orders.groupby(['Fund', 'CNPJ', 'Asset', 'Side']).agg({
        'Qty_Allocation': 'sum'
    }).reset_index()
    
    # Get current date
    current_date = date.today().strftime("%Y-%m-%d")
    
    # Create final allocation DataFrame in new format
    final_allocation = pd.DataFrame({
        'trade_date': current_date,
        'asset': grouped['Asset'],
        'side': grouped['Side'].map({'BUY': 'C', 'SELL': 'V'}),
        'asset_amount': grouped['Qty_Allocation'].astype(int),
        'exchange_code': 'XP (3)',  # Fixed broker code
        'fund': grouped['Fund'],
        'cnpj': grouped['CNPJ']
    })
    
    # Sort by Asset, then Side, then Fund for better organization
    final_allocation = final_allocation.sort_values(['asset', 'side', 'fund']).reset_index(drop=True)
    
    print(f"Created {len(final_allocation)} total allocation orders")
    
    return final_allocation

def save_lot_units(df: pd.DataFrame, assets_list: List[str]) -> pd.DataFrame:
    """
    Save lot units to BigQuery after filtering and processing the data.
    
    Args:
        df: DataFrame with lot unit data
        assets_list: List of assets to filter
        
    Returns:
        DataFrame with processed and saved data
    """
    try:
        # Validate inputs
        if df.empty:
            raise ValueError("Input DataFrame is empty")
        if not assets_list:
            raise ValueError("Assets list is empty")
            
        # Filter and rename columns
        df = df[["Fund", "CNPJ", "Asset", "Side", "Qty_Primary"]]
        
        df_assets = df[(df["Asset"].isin(assets_list)) & (df["Qty_Primary"] != 0)].rename(columns={
            "Fund": "fund_name",
            "CNPJ": "fund_cnpj",
            "Asset": "asset",
            "Side": "side",
            "Qty_Primary": "primary_units"
        })

        if df_assets.empty:
            print("No records to save after filtering")
            return df_assets

        # Add request_date column
        current_date = date.today().strftime("%Y-%m-%d")
        df_assets["request_date"] = current_date
        list_sentences = create_primary_sentences(df_assets)
        print(list_sentences)
        # Define BigQuery parameters
        project_id = "hdx-data-platform-users"
        table_id = "team_trading.primary_requests"
        
        # Initialize BigQuery client
        client = bigquery.Client(project=project_id)
        
        # Delete existing records for each fund_cnpj
        for fund_cnpj in df_assets['fund_cnpj'].unique():
            delete_query = f"""
            DELETE FROM `{project_id}.{table_id}`
            WHERE request_date = '{current_date}'
            AND fund_cnpj = '{fund_cnpj}'
            """
            # Execute delete query
            query_job = client.query(delete_query)
            query_job.result()  # Wait for query to complete
        
        # Upload to BigQuery
        pandas_gbq.to_gbq(
            df_assets,
            destination_table=table_id,
            project_id=project_id,
            if_exists="append",
            table_schema=[
                {"name": "request_date", "type": "DATE"},
                {"name": "fund_name", "type": "STRING"},
                {"name": "fund_cnpj", "type": "STRING"},
                {"name": "asset", "type": "STRING"},
                {"name": "side", "type": "STRING"},
                {"name": "primary_units", "type": "FLOAT"},
            ],
            api_method="load_csv"
        )
        
        print(f"Successfully saved {len(df_assets)} records to BigQuery")
        return df_assets, list_sentences
        
    except Exception as e:
        print(f"Error saving lot units to BigQuery: {str(e)}")
        raise

def save_secondary_allocation_trades(df: pd.DataFrame, 
                                   csv_folder_path: str = r"G:\Drives compartilhados\Ops-Trading\Onshore\CSV B3 gestora - Executed Trades by fund") -> pd.DataFrame:
    """
    Save secondary allocation trades to BigQuery and CSV file
    
    Args:
        df: DataFrame with secondary allocation data (from create_final_allocation_for_broker)
        csv_folder_path: Path to save CSV file (optional, has default)
        
    Returns:
        DataFrame with processed and saved data
    """
    try:
        # Validate inputs
        if df.empty:
            raise ValueError("Input DataFrame is empty")
            
        # Get current date
        current_date = date.today().strftime("%Y-%m-%d")
        
        # === SAVE TO BIGQUERY ===
        # Rename trade_date to request_date to match BigQuery table
        df_to_save = df.copy()
        if 'trade_date' in df_to_save.columns:
            df_to_save = df_to_save.rename(columns={'trade_date': 'request_date'})
        
        # Define BigQuery parameters
        project_id = "hdx-data-platform-users"
        table_id = "team_trading.secondary_allocation_trades"
        
        # Initialize BigQuery client
        client = bigquery.Client(project=project_id)
        
        # Delete existing records for each fund on the same date
        for fund_name in df_to_save['fund'].unique():
            delete_query = f"""
            DELETE FROM `{project_id}.{table_id}`
            WHERE request_date = '{current_date}'
            AND fund = '{fund_name}'
            """
            # Execute delete query
            query_job = client.query(delete_query)
            query_job.result()  # Wait for query to complete
        
        # Upload to BigQuery
        pandas_gbq.to_gbq(
            df_to_save,
            destination_table=table_id,
            project_id=project_id,
            if_exists="append",
            table_schema=[
                {"name": "request_date", "type": "DATE"},
                {"name": "asset", "type": "STRING"},
                {"name": "side", "type": "STRING"},
                {"name": "asset_amount", "type": "BIGNUMERIC"},
                {"name": "exchange_code", "type": "STRING"},
                {"name": "fund", "type": "STRING"},
                {"name": "cnpj", "type": "STRING"},
            ],
            api_method="load_csv"
        )
        
        print(f"Successfully saved {len(df_to_save)} records to BigQuery")
        
        # === SAVE TO CSV ===
        # Create filename
        filename = f"aloc_trading_{current_date}.csv"
        
        # Full file path
        full_path = os.path.join(csv_folder_path, filename)
        
        # Save original DataFrame to CSV (with trade_date column)
        df.to_csv(full_path, index=False)
        
        print(f"Successfully saved CSV to: {full_path}")
        
        return df_to_save
        
    except Exception as e:
        print(f"Error saving secondary allocation trades: {str(e)}")
        raise

def execute_daily_trading_workflow(step: str = "all", max_adjustment_iterations: int = 3):
    """
    Execute specific parts of the daily trading workflow
    
    Args:
        step: Which step to run - "initial", "primary", "adjustments", or "all"
        max_adjustment_iterations: Max adjustment cycles to run
        
    Returns:
        dict: Results from the executed step(s)
    """
    
    results = {}
    
    if step in ["initial", "all"]:
        print("=== Step 1: Initial Orders ===")
        simple_order, rebalance_order, sentences, df_grouped_simple_order, df_grouped_rebalance_order, trades_data, on_shore_trades, off_shore_trades = get_and_organize_trades()
        
        results['initial_orders'] = {
            'sentences': sentences,
            'on_shore_trades': on_shore_trades,
            'off_shore_trades': off_shore_trades,
            'grouped_simple': df_grouped_simple_order,
            'grouped_rebalance': df_grouped_rebalance_order
        }
        
        print(f"Generated {len(sentences)} initial order sentences")
        # Here you would send the email
        # send_trading_email(sentences, ...)
        
        if step == "initial":
            return results
    
    if step in ["primary", "all"]:
        print("\n=== Step 2: Primary Units Calculation ===")
        
        # Get on_shore_trades from previous step or reload
        if 'initial_orders' in results:
            on_shore_trades = results['initial_orders']['on_shore_trades']
        else:
            # Reload if running this step independently
            _, _, _, _, _, _, on_shore_trades, _ = get_and_organize_trades()
        
        trades_df = TradingAPIManager('hdx-routines-secret').get_consolidated_trades()
        result_df_for_primary_save = TradeAllocationCalculator(trades_df, on_shore_trades).calculate_primary_units_w_executed_trades()
        saved_primary_data, primary_sentences = save_lot_units(result_df_for_primary_save, list_nasdaq)
        
        results['primary_data'] = {
            'saved_data': saved_primary_data,
            'sentences': primary_sentences,
            'allocation_df': result_df_for_primary_save
        }
        
        print("Primary units calculated and saved")
        
        if step == "primary":
            return results
    
    if step in ["adjustments", "all"]:
        print("\n=== Step 3: Adjustment Cycles ===")
        
        # Get on_shore_trades from previous step or reload
        if 'initial_orders' in results:
            on_shore_trades = results['initial_orders']['on_shore_trades']
        else:
            # Reload if running this step independently
            _, _, _, _, _, _, on_shore_trades, _ = get_and_organize_trades()
        
        trades_df = TradingAPIManager('hdx-routines-secret').get_consolidated_trades()
        calculator = TradeAllocationCalculator(trades_df, on_shore_trades)
        adjustment_results = calculator.run_adjustment_cycle(max_iterations=max_adjustment_iterations)
        
        results['adjustment_results'] = adjustment_results
        
        print(f"Completed {len(adjustment_results)} adjustment iterations")
    
    return results

# Also add these convenient wrapper functions:
def run_initial_orders():
    """Run only the initial orders step"""
    return execute_daily_trading_workflow(step="initial")

def run_primary_calculation():
    """Run only the primary units calculation step"""
    return execute_daily_trading_workflow(step="primary")

def run_adjustment_cycle(max_iterations: int = 3):
    """Run only the adjustment cycles step"""
    return execute_daily_trading_workflow(step="adjustments", max_adjustment_iterations=max_iterations)

def complete_adjustment_workflow(max_iterations: int = 3):
    """Run complete adjustment workflow and return broker-ready orders"""
    
    # Run adjustment cycles
    print("Running adjustment cycles...")
    adjustment_results = run_adjustment_cycle(max_iterations)
    
    # Debug: Check what we got back
    print(f"Type of adjustment_results: {type(adjustment_results)}")
    print(f"Content: {adjustment_results}")
    
    if not adjustment_results:
        print("No adjustments were needed")
        return pd.DataFrame()
    
    # Handle different return types
    if isinstance(adjustment_results, list):
        # If it's a list, get the last iteration
        final_allocation = adjustment_results[-1]['full_allocation']
    elif isinstance(adjustment_results, dict):
        # If it's a dict, it might have 'adjustment_results' key
        if 'adjustment_results' in adjustment_results:
            iterations = adjustment_results['adjustment_results']
            final_allocation = iterations[-1]['full_allocation']
        else:
            # Or it might be the result directly
            final_allocation = adjustment_results.get('full_allocation')
            if final_allocation is None:
                print("Could not find allocation data in results")
                return pd.DataFrame()
    else:
        print(f"Unexpected result type: {type(adjustment_results)}")
        return pd.DataFrame()
    
    # Create broker orders
    print("\nCreating final broker orders...")
    broker_orders = create_final_allocation_for_broker(final_allocation)
    
    return {
        'adjustment_history': adjustment_results,
        'broker_orders': broker_orders
    }

def check_adjustment_status():
    """Quick check to see current adjustment status without running cycles"""
    _, _, _, _, _, _, on_shore_trades, _ = get_and_organize_trades()
    trades_df = TradingAPIManager('hdx-routines-secret').get_consolidated_trades()
    calculator = TradeAllocationCalculator(trades_df, on_shore_trades)
    
    # Just get current allocation without running cycles
    allocation_results = calculator.calculate_update_secondary_values()
    email_body = create_adjustment_email_sentences(allocation_results, trades_df)
    
    print("Current adjustment status:")
    print(email_body)
    
    return allocation_results


# run_initial_orders()

# run_primary_calculation()    

# run_adjustment_cycle(int = 3)


# check_adjustment_status()
# xyz = get_and_organize_trades()

In [108]:
# run_initial_orders()

# run_primary_calculation()    

# run_adjustment_cycle(1)

result = complete_adjustment_workflow(1)
broker_orders = result['broker_orders']

save_secondary_allocation_trades(broker_orders)
# save_broker_orders(result['broker_orders'])
broker_orders

Running adjustment cycles...

=== Step 3: Adjustment Cycles ===
                                  Fund                 CNPJ   Asset  Side  \
0  Hashdex 100 Nasdaq Crypto Index FIM   33.736.845/0001-07    NCIQ  SELL   
1                          HDA I - FIM   33.736.782/0001-80  HASH11   BUY   
2                          HC II - FIM   35.001.472/0001-43  HASH11   BUY   
3                        HBTC I FIM IE   38.167.819/0001-10  BITH11  SELL   
4       Hashdex Crypto Long Biased FIM   47.612.160/0001-55  HASH11   BUY   
5         Hashdex Crypto Selection FIM   45.604.913/0001-55  SOLH11  SELL   

      Amount     Strategy  
0   100490.0  SimpleOrder  
1   674000.0  SimpleOrder  
2   888000.0  SimpleOrder  
3  1290000.0  SimpleOrder  
4    16100.0  SimpleOrder  
5   490000.0  SimpleOrder  
['Favor iniciar uma venda de BITH11 até R$ 1.290.000,00, até 30 bps abaixo do justo', 'Favor iniciar uma compra de HASH11 até R$ 1.578.100,00, até 30 bps acima do justo', 'Favor iniciar uma venda de N

c:\Users\lucas.afonso_hashdex\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
1it [00:04,  4.88s/it]

Successfully saved 5 records to BigQuery
Successfully saved CSV to: G:\Drives compartilhados\Ops-Trading\Onshore\CSV B3 gestora - Executed Trades by fund\aloc_trading_2025-06-04.csv


,trade_date,asset,side,asset_amount,exchange_code,fund,cnpj
0,2025-06-04,BITH11,V,1994,XP (3),HBTC I FIM IE,38.167.819/0001-10
1,2025-06-04,HASH11,C,11316,XP (3),HC II - FIM,35.001.472/0001-43
2,2025-06-04,HASH11,C,8589,XP (3),HDA I - FIM,33.736.782/0001-80
3,2025-06-04,HASH11,C,205,XP (3),Hashdex Crypto Long Biased FIM,47.612.160/0001-55
4,2025-06-04,SOLH11,V,16714,XP (3),Hashdex Crypto Selection FIM,45.604.913/0001-55


In [107]:
result['broker_orders']

,ATIVO,(C/V),QUANTIDADE,COD. CORRETORA,Fundo,CNPJ
0,BITH11,V,1994,XP (3),HBTC I FIM IE,38.167.819/0001-10
1,HASH11,C,11316,XP (3),HC II - FIM,35.001.472/0001-43
2,HASH11,C,8589,XP (3),HDA I - FIM,33.736.782/0001-80
3,HASH11,C,205,XP (3),Hashdex Crypto Long Biased FIM,47.612.160/0001-55
4,SOLH11,V,16714,XP (3),Hashdex Crypto Selection FIM,45.604.913/0001-55


## teste from here

In [83]:
simple_order, rebalance_order, sentences, df_grouped_simple_order, df_grouped_rebalance_order, trades_data, on_shore_trades, off_shore_trades = get_and_organize_trades()
on_shore_trades


                                  Fund                 CNPJ   Asset  Side  \
0  Hashdex 100 Nasdaq Crypto Index FIM   33.736.845/0001-07    NCIQ  SELL   
1                          HDA I - FIM   33.736.782/0001-80  HASH11   BUY   
2                          HC II - FIM   35.001.472/0001-43  HASH11   BUY   
3                        HBTC I FIM IE   38.167.819/0001-10  BITH11  SELL   
4       Hashdex Crypto Long Biased FIM   47.612.160/0001-55  HASH11   BUY   
5         Hashdex Crypto Selection FIM   45.604.913/0001-55  SOLH11  SELL   

      Amount     Strategy  
0   100490.0  SimpleOrder  
1   674000.0  SimpleOrder  
2   888000.0  SimpleOrder  
3  1290000.0  SimpleOrder  
4    16100.0  SimpleOrder  
5   490000.0  SimpleOrder  
['Favor iniciar uma venda de BITH11 até R$ 1.290.000,00, até 30 bps abaixo do justo', 'Favor iniciar uma compra de HASH11 até R$ 1.578.100,00, até 30 bps acima do justo', 'Favor iniciar uma venda de NCIQ até R$ 100.490,00, até 30 bps abaixo do justo', 'Favor inici

,Fund,CNPJ,Asset,Side,Amount,Strategy
1,HDA I - FIM,33.736.782/0001-80,HASH11,BUY,674000.0,SimpleOrder
2,HC II - FIM,35.001.472/0001-43,HASH11,BUY,888000.0,SimpleOrder
3,HBTC I FIM IE,38.167.819/0001-10,BITH11,SELL,1290000.0,SimpleOrder
4,Hashdex Crypto Long Biased FIM,47.612.160/0001-55,HASH11,BUY,16100.0,SimpleOrder
5,Hashdex Crypto Selection FIM,45.604.913/0001-55,SOLH11,SELL,490000.0,SimpleOrder


In [76]:
on_shore_trades

,Fund,CNPJ,Asset,Side,Amount,Strategy
1,HDA I - FIM,33.736.782/0001-80,HASH11,BUY,674000.0,SimpleOrder
2,HC II - FIM,35.001.472/0001-43,HASH11,BUY,888000.0,SimpleOrder
3,HBTC I FIM IE,38.167.819/0001-10,BITH11,SELL,1290000.0,SimpleOrder
4,Hashdex Crypto Long Biased FIM,47.612.160/0001-55,HASH11,BUY,16100.0,SimpleOrder
5,Hashdex Crypto Selection FIM,45.604.913/0001-55,SOLH11,SELL,490000.0,SimpleOrder


In [ ]:

def send_trading_email(sentences, subject, sender_email, sender_password, recipient_emails):
    """
    Send email using Outlook/Hotmail
    
    Parameters:
    sentences: list of trading instruction sentences
    subject: email subject  
    sender_email: your outlook email (e.g., "you@outlook.com" or "you@hotmail.com")
    sender_password: your app password or regular password
    recipient_emails: list of recipient emails
    """
    try:
        # Create message
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = ', '.join(recipient_emails)
        msg['Subject'] = subject
        
        # Create email body
        body = f"Trading Instructions - {datetime.now().strftime('%Y-%m-%d %H:%M')}\n\n"
        for i, sentence in enumerate(sentences, 1):
            body += f"{i}. {sentence}\n\n"
        
        msg.attach(MIMEText(body, 'plain'))
        
        # Outlook SMTP settings
        server = smtplib.SMTP('smtp-mail.outlook.com', 587)
        server.starttls()
        server.login(sender_email, sender_password)
        server.send_message(msg)
        server.quit()
        
        print(f"Email sent successfully to {', '.join(recipient_emails)}")
        
    except Exception as e:
        print(f"Error sending email: {str(e)}")

# Usage example:
send_trading_email(
    sentences=sentences,
    subject="Daily Trading Instructions", 
    sender_email="lucas.afonso@hashdex.com",
    sender_password="pedro.nacif@hashdex.com",
    recipient_emails=["pedro.nacif@hashdex.com"]
)

In [75]:
# trades = get_and_organize_trades()
# trades[2]


# executed_trades_csv = pd.read_csv(r'G:\Drives compartilhados\Investment Management\Trading_Routines_logics\trades_executed.csv')
# executed_trades_csv
# trades[0]

In [ ]:
## frist input to primary

# save_lot_units(result_df, list_thematics)
trades_df = TradingAPIManager('hdx-routines-secret').get_consolidated_trades()
result_df_for_primary_save = TradeAllocationCalculator(trades_df, on_shore_trades).calculate_primary_units_w_executed_trades()
save_lot_units(result_df_for_primary_save, list_nasdaq)

In [74]:
## second input to adjust secondary
trades_df_afterward = TradingAPIManager('hdx-routines-secret').get_consolidated_trades()
allocation_to_adjust = TradeAllocationCalculator(trades_df_afterward, on_shore_trades).calculate_update_secondary_values()
allocation_to_adjust

  request_date                     fund_name            fund_cnpj   asset  \
0   2025-06-04                 HBTC I FIM IE   38.167.819/0001-10  BITH11   
1   2025-06-04  Hashdex Crypto Selection FIM   45.604.913/0001-55  SOLH11   

   side primary_units  
0  SELL             1  
1  SELL             1  
      ETF  Lot Units  Estimated Opening iNAV  iNAV per Share  \
0  HASH11    15000.0                    83.0       79.357965   

   Estimated Opening Amount  Current Lot Amount  
0                 1245000.0        1.190369e+06  
Final squared error: 6.888164482958227e-07
Total shares executed: 20110
Total primary units used: 0.0
      ETF  Lot Units  Estimated Opening iNAV  iNAV per Share  \
1  BITH11     7500.0                   141.0      135.503145   

   Estimated Opening Amount  Current Lot Amount  
1                 1057500.0        1.016274e+06  
Final squared error: 1.0254961382071396e-05
Total shares executed: 1994
Total primary units used: 1
      ETF  Lot Units  Estimated Open

,Fund,CNPJ,Asset,Side,Qty_Allocation,Qty_Primary,Price_Executed,Financial_Allocation,Financial_Primary,Target_Value,Target_to_Adjustment,Error
3,HBTC I FIM IE,38.167.819/0001-10,BITH11,SELL,1994,1.0,135.203310,269595.400000,1.016274e+06,1290000.0,4131.014553,1.025496e-05
1,HC II - FIM,35.001.472/0001-43,HASH11,BUY,11316,0.0,78.471430,887982.703399,0.000000e+00,888000.0,17.296601,3.793985e-10
0,HDA I - FIM,33.736.782/0001-80,HASH11,BUY,8589,0.0,78.471430,673991.113423,0.000000e+00,674000.0,8.886577,1.738398e-10
2,Hashdex Crypto Long Biased FIM,47.612.160/0001-55,HASH11,BUY,205,0.0,78.471430,16086.643178,0.000000e+00,16100.0,13.356822,6.882632e-07
4,Hashdex Crypto Selection FIM,45.604.913/0001-55,SOLH11,SELL,16714,1.0,24.470263,408995.980000,6.139661e+04,490000.0,19607.405350,1.601209e-03


In [99]:


# check_adjustment_status()

Running adjustment cycles...

=== Step 3: Adjustment Cycles ===
                                  Fund                 CNPJ   Asset  Side  \
0  Hashdex 100 Nasdaq Crypto Index FIM   33.736.845/0001-07    NCIQ  SELL   
1                          HDA I - FIM   33.736.782/0001-80  HASH11   BUY   
2                          HC II - FIM   35.001.472/0001-43  HASH11   BUY   
3                        HBTC I FIM IE   38.167.819/0001-10  BITH11  SELL   
4       Hashdex Crypto Long Biased FIM   47.612.160/0001-55  HASH11   BUY   
5         Hashdex Crypto Selection FIM   45.604.913/0001-55  SOLH11  SELL   

      Amount     Strategy  
0   100490.0  SimpleOrder  
1   674000.0  SimpleOrder  
2   888000.0  SimpleOrder  
3  1290000.0  SimpleOrder  
4    16100.0  SimpleOrder  
5   490000.0  SimpleOrder  
['Favor iniciar uma venda de BITH11 até R$ 1.290.000,00, até 30 bps abaixo do justo', 'Favor iniciar uma compra de HASH11 até R$ 1.578.100,00, até 30 bps acima do justo', 'Favor iniciar uma venda de N

In [104]:
save_broker_orders(result['broker_orders'])

# # Or if you have the DataFrame directly
# save_broker_orders(broker_orders)

Broker orders saved to: G:\Drives compartilhados\Ops-Trading\Onshore\CSV B3 gestora - Executed Trades by fund\aloc_trading_2025-06-04.csv


In [ ]:
from google.cloud import bigquery
from typing import Optional

def truncate_primary_requests_table() -> None:
    """
    Deletes all records from the primary_requests table in BigQuery.
    
    Returns:
        None
    
    Raises:
        Exception: If the deletion operation fails
    """
    try:
        # Define BigQuery parameters
        project_id = "hdx-data-platform-users"
        table_id = "team_trading.primary_requests"
        
        # Initialize BigQuery client
        client = bigquery.Client(project=project_id)
        
        # Create delete query
        delete_query = f"""
        DELETE FROM `{project_id}.{table_id}`
        WHERE TRUE
        """
        
        # Execute query
        query_job = client.query(delete_query)
        query_job.result()  # Wait for query to complete
        
        print(f"Successfully deleted all records from {table_id}")
        
    except Exception as e:
        print(f"Error truncating table: {str(e)}")
        raise
    
    
truncate_primary_requests_table()


In [93]:

allocation_results = TradeAllocationCalculator(trades_df, on_shore_trades).calculate_update_secondary_values()

# Create final consolidated allocation for broker
broker_orders = create_final_allocation_for_broker(allocation_results)

  request_date                     fund_name            fund_cnpj   asset  \
0   2025-06-04                 HBTC I FIM IE   38.167.819/0001-10  BITH11   
1   2025-06-04  Hashdex Crypto Selection FIM   45.604.913/0001-55  SOLH11   

   side primary_units  
0  SELL             1  
1  SELL             1  
      ETF  Lot Units  Estimated Opening iNAV  iNAV per Share  \
0  HASH11    15000.0                    83.0        79.05946   

   Estimated Opening Amount  Current Lot Amount  
0                 1245000.0        1.185892e+06  
Final squared error: 6.888164482958227e-07
Total shares executed: 20110
Total primary units used: 0.0
      ETF  Lot Units  Estimated Opening iNAV  iNAV per Share  \
1  BITH11     7500.0                   141.0      135.094868   

   Estimated Opening Amount  Current Lot Amount  
1                 1057500.0        1.013212e+06  
Zero or invalid inputs detected. Returning zero allocation.


C:\Users\lucas.afonso_hashdex\AppData\Local\Temp\ipykernel_15896\3287436703.py:49: RuntimeWarning: divide by zero encountered in divide
  self.target_values / self.price_executed
C:\Users\lucas.afonso_hashdex\AppData\Local\Temp\ipykernel_15896\3287436703.py:50: RuntimeWarning: invalid value encountered in cast
  ).astype(int)


      ETF  Lot Units  Estimated Opening iNAV  iNAV per Share  \
3  SOLH11     2500.0               25.250982       24.398015   

   Estimated Opening Amount  Current Lot Amount  
3              63127.455625        60995.037867  
Final squared error: 0.0016674692461237115
Total shares executed: 16714
Total primary units used: 1
Created 4 total allocation orders


In [94]:
broker_orders

,ATIVO,(C/V),QUANTIDADE,COD. CORRETORA,Fundo,CNPJ
0,HASH11,C,11316,XP (3),HC II - FIM,35.001.472/0001-43
1,HASH11,C,8589,XP (3),HDA I - FIM,33.736.782/0001-80
2,HASH11,C,205,XP (3),Hashdex Crypto Long Biased FIM,47.612.160/0001-55
3,SOLH11,V,16714,XP (3),Hashdex Crypto Selection FIM,45.604.913/0001-55


### functions to create python scripts

In [87]:
class TradeAllocationCalculator:
    """Calculator for trade allocations with primary units and adjustment orders"""
    
    def __init__(self, executed_trades: pd.DataFrame, df_funds_target: pd.DataFrame):
        """
        Initialize calculator with trade data
        
        Args:
            executed_trades: DataFrame with executed trades
            df_funds_target: DataFrame with fund targets
        """
        self.executed_trades = executed_trades
        self.df_funds_target = df_funds_target
        self.side_map = {'BUY': 'C', 'SELL': 'V'}
        
    def _process_single_trade(self, asset: str, side: str) -> pd.DataFrame:
        """Process a single trade allocation and return allocation DataFrame"""
        # Get target and executed trades
        target_df = self.df_funds_target[(self.df_funds_target['Asset'] == asset) & 
                                    (self.df_funds_target['Side'] == side)]
        
        # If no targets for this asset/side combination, return empty DataFrame
        if target_df.empty:
            return pd.DataFrame()
        
        # Get executed trades if any
        executed = self.executed_trades[(self.executed_trades['SIDE'] == self.side_map[side]) & 
                                    (self.executed_trades['ATIVO'] == asset)]
        
        # Set execution metrics - use zeros if no executions
        if executed.empty:
            qty_executed = 0
            price_executed = 0
        else:
            qty_executed = executed['QTY. AVAILABLE'].sum()
            price_executed = executed['AVG_PRICE'].mean()
        
        # Prepare solver input
        funds_target = target_df[["Fund", "Amount"]].rename(columns={
            "Fund": "fund", 
            "Amount": "value_target"
        })
        
        # Create fund name and CNPJ mapping
        fund_names = target_df[["Fund"]].to_dict()['Fund']
        fund_cnpjs = target_df.set_index('Fund')['CNPJ'].to_dict()
        
        # Run solver
        try:
            solver_result = AllocationSolver(
                qty_executed=qty_executed,
                price_executed=price_executed,
                asset_trade=asset,
                df_funds_target=funds_target,
                max_iterations=10000
            ).solve()
            
            # Calculate financial values
            financial_allocation = solver_result['financial_allocation']
            financial_primary = solver_result['financial_primary_units']
            target_value = solver_result['value_target']
            
            # Create result DataFrame
            result_df = pd.DataFrame({
                'Fund': solver_result.index.map(lambda x: fund_names.get(x, x)),
                'CNPJ': solver_result.index.map(lambda x: fund_cnpjs.get(fund_names.get(x, x), '')),
                'Asset': asset,
                'Side': side,
                'Qty_Allocation': solver_result['qty_allocation'],
                'Qty_Primary': solver_result['qty_primary_units'],
                'Price_Executed': price_executed,
                'Financial_Allocation': financial_allocation,
                'Financial_Primary': financial_primary,
                'Target_Value': target_value,
                'Target_to_Adjustment': target_value - (financial_allocation + financial_primary),
                'Error': solver_result['Squared_error'],
                'Strategy': 'MainOrder'  # Mark as main order
            })
            
            # Order columns
            columns = ['Fund', 'CNPJ', 'Asset', 'Side', 'Qty_Allocation', 'Qty_Primary', 
                    'Price_Executed', 'Financial_Allocation', 'Financial_Primary',
                    'Target_Value', 'Target_to_Adjustment', 'Error', 'Strategy']
            result_df = result_df[columns]
            
            return result_df
            
        except Exception as e:
            print(f"Error processing {asset} {side}: {str(e)}")
            return pd.DataFrame()
    
    def calculate_primary_units_w_executed_trades(self) -> pd.DataFrame:
        """Calculate all allocations and return single consolidated DataFrame"""
        all_results = []
        
        for asset in self.df_funds_target['Asset'].unique():
            # Process both buy and sell sides
            for side in ['BUY', 'SELL']:
                result = self._process_single_trade(asset, side)
                if not result.empty:
                    all_results.append(result)
        
        # Return empty DataFrame if no results
        if not all_results:
            return pd.DataFrame(columns=['Fund', 'CNPJ', 'Asset', 'Side', 'Qty_Allocation', 'Qty_Primary', 
                                       'Price_Executed', 'Financial_Allocation', 'Financial_Primary',
                                       'Target_Value', 'Target_to_Adjustment', 'Error', 'Strategy'])
        
        # Combine all results and sort
        final_result = pd.concat(all_results, ignore_index=True).sort_values(
            by=['Asset', 'Side', 'Fund']
        )
        
        return final_result
    
    def get_summary(self) -> pd.DataFrame:
        """Get summary of allocations by Asset and Side"""
        result = self.calculate_primary_units_w_executed_trades()
        if result.empty:
            return pd.DataFrame()
            
        summary = result.groupby(['Asset', 'Side']).agg({
            'Qty_Allocation': 'sum',
            'Qty_Primary': 'sum',
            'Financial_Allocation': 'sum',
            'Financial_Primary': 'sum',
            'Target_Value': 'sum',
            'Target_to_Adjustment': 'sum',
            'Error': 'mean'
        }).round(2)
        
        return summary

    def _create_adjustment_orders(self, main_results: pd.DataFrame) -> pd.DataFrame:
        """Create adjustment orders for negative Target_to_Adjustment values"""
        # Filter for negative adjustments (over-allocations)
        negative_adjustments = main_results[main_results['Target_to_Adjustment'] < 0].copy()
        
        if negative_adjustments.empty:
            return pd.DataFrame()
        
        adjustment_orders = []
        
        for _, row in negative_adjustments.iterrows():
            # Calculate adjustment quantity (convert financial amount to shares)
            adjustment_financial = abs(row['Target_to_Adjustment'])
            adjustment_qty = 0
            
            if row['Price_Executed'] > 0:
                adjustment_qty = int(adjustment_financial / row['Price_Executed'])
            
            # Skip if adjustment quantity is 0
            if adjustment_qty == 0:
                continue
            
            # Determine opposite side
            opposite_side = 'SELL' if row['Side'] == 'BUY' else 'BUY'
            
            # Create adjustment order
            adjustment_order = {
                'Fund': row['Fund'],
                'CNPJ': row['CNPJ'],
                'Asset': row['Asset'],
                'Side': opposite_side,
                'Qty_Allocation': adjustment_qty,
                'Qty_Primary': 0,  # Adjustments are secondary only
                'Price_Executed': row['Price_Executed'],
                'Financial_Allocation': adjustment_qty * row['Price_Executed'],
                'Financial_Primary': 0,
                'Target_Value': 0,  # No target for adjustment orders
                'Target_to_Adjustment': 0,  # Should be zero after adjustment
                'Error': 0,
                'Strategy': 'AdjustmentOrder'
            }
            
            adjustment_orders.append(adjustment_order)
        
        if adjustment_orders:
            return pd.DataFrame(adjustment_orders)
        else:
            return pd.DataFrame()
    
    def calculate_update_secondary_values(self) -> pd.DataFrame:
        """Calculate allocations with fixed primary units and create adjustment orders"""
        current_date = date.today().strftime("%Y-%m-%d")
        
        query = f"""
        SELECT 
            *
        FROM 
            `hdx-data-platform-users.team_trading.primary_requests`
        WHERE 
            request_date = '{current_date}'
        """
        
        df_primary_requests = client.query(query).to_dataframe()
        print(df_primary_requests)
        
        # Process allocations with fixed primary units
        all_results = []
        
        for asset in self.df_funds_target['Asset'].unique():
            # Process both buy and sell sides
            for side in ['BUY', 'SELL']:
                # Get target and executed trades
                target_df = self.df_funds_target[(self.df_funds_target['Asset'] == asset) & 
                                            (self.df_funds_target['Side'] == side)]
                
                # If no targets for this asset/side combination, skip
                if target_df.empty:
                    continue
                
                # Get executed trades if any
                executed = self.executed_trades[(self.executed_trades['SIDE'] == self.side_map[side]) & 
                                            (self.executed_trades['ATIVO'] == asset)]
                
                # Set execution metrics - use zeros if no executions
                if executed.empty:
                    qty_executed = 0
                    price_executed = 0
                else:
                    qty_executed = executed['QTY. AVAILABLE'].sum()
                    price_executed = executed['AVG_PRICE'].mean()
                
                # Prepare solver input - INCLUDE CNPJ COLUMN
                funds_target = target_df[["Fund", "CNPJ", "Amount"]].rename(columns={
                    "Fund": "fund", 
                    "CNPJ": "cnpj",
                    "Amount": "value_target"
                })
                
                # Create fund name and CNPJ mapping
                fund_names = target_df[["Fund"]].to_dict()['Fund']
                fund_cnpjs = target_df.set_index('Fund')['CNPJ'].to_dict()
                
                # Run solver with fixed primary units
                try:
                    solver_result = AllocationSolver(
                        qty_executed=qty_executed,
                        price_executed=price_executed,
                        asset_trade=asset,
                        df_funds_target=funds_target,
                        max_iterations=10000
                    ).solve_with_fixed_primary(df_primary_requests)
                    
                    # Calculate financial values
                    financial_allocation = solver_result['financial_allocation']
                    financial_primary = solver_result['financial_primary_units']
                    target_value = solver_result['value_target']
                    
                    # Create result DataFrame
                    result_df = pd.DataFrame({
                        'Fund': solver_result.index.map(lambda x: fund_names.get(x, x)),
                        'CNPJ': solver_result.index.map(lambda x: fund_cnpjs.get(fund_names.get(x, x), '')),
                        'Asset': asset,
                        'Side': side,
                        'Qty_Allocation': solver_result['qty_allocation'],
                        'Qty_Primary': solver_result['qty_primary_units'],
                        'Price_Executed': price_executed,
                        'Financial_Allocation': financial_allocation,
                        'Financial_Primary': financial_primary,
                        'Target_Value': target_value,
                        'Target_to_Adjustment': target_value - (financial_allocation + financial_primary),
                        'Error': solver_result['Squared_error'],
                        'Strategy': 'MainOrder'
                    })
                    
                    # Order columns
                    columns = ['Fund', 'CNPJ', 'Asset', 'Side', 'Qty_Allocation', 'Qty_Primary', 
                            'Price_Executed', 'Financial_Allocation', 'Financial_Primary',
                            'Target_Value', 'Target_to_Adjustment', 'Error', 'Strategy']
                    result_df = result_df[columns]
                    
                    all_results.append(result_df)
                    
                except Exception as e:
                    print(f"Error processing {asset} {side}: {str(e)}")
                    continue
        
        # Combine main results
        if not all_results:
            main_results = pd.DataFrame(columns=['Fund', 'CNPJ', 'Asset', 'Side', 'Qty_Allocation', 'Qty_Primary', 
                                               'Price_Executed', 'Financial_Allocation', 'Financial_Primary',
                                               'Target_Value', 'Target_to_Adjustment', 'Error', 'Strategy'])
        else:
            main_results = pd.concat(all_results, ignore_index=True)
        
        # Create adjustment orders for negative Target_to_Adjustment
        adjustment_orders = self._create_adjustment_orders(main_results)
        
        # Combine main orders and adjustment orders
        if not adjustment_orders.empty:
            final_result = pd.concat([main_results, adjustment_orders], ignore_index=True).sort_values(
                by=['Asset', 'Side', 'Fund', 'Strategy']
            )
        else:
            final_result = main_results.sort_values(by=['Asset', 'Side', 'Fund'])
        
        return final_result
    
    
    def run_adjustment_cycle(self, max_iterations: int = 3):
        """Run the complete adjustment cycle"""
        all_iterations = []
        
        for iteration in range(1, max_iterations + 1):
            print(f"\n=== Adjustment Iteration {iteration} ===")
            
            # Get updated trades
            updated_trades = TradingAPIManager('hdx-routines-secret').get_consolidated_trades()
            self.executed_trades = updated_trades  # Update with latest trades
            
            # Calculate adjustments
            allocation_results = self.calculate_update_secondary_values()
            
            # Create comprehensive email with tables
            email_body = create_adjustment_email_sentences(allocation_results, updated_trades)
            
            # Check if any orders needed (simple check)
            has_orders = "Nenhuma ordem de ajuste necessária" not in email_body
            
            if not has_orders:
                print("No more adjustments needed")
                break
                
            # Store results
            iteration_data = {
                'iteration': iteration,
                'email_body': email_body,
                'full_allocation': allocation_results
            }
            all_iterations.append(iteration_data)
            
            print("Generated adjustment email:")
            print(email_body)
                
            # Here you would send email and wait for execution
            
        return all_iterations

In [72]:
def create_zero_result(df_funds_target: pd.DataFrame, lot_amount: float) -> pd.DataFrame:
    """Create a result DataFrame with zero allocations"""
    df_result = df_funds_target.copy()
    df_result["qty_allocation"] = 0
    df_result["qty_allocation_rounded"] = 0
    df_result["financial_allocation"] = 0
    df_result["target_to_primary"] = df_result["value_target"]
    df_result["qty_primary_units"] = df_result["value_target"].apply(
        lambda x: calculate_primary_request(x, lot_amount)
    )
    df_result["financial_primary_units"] = df_result["qty_primary_units"] * lot_amount
    df_result["Squared_error"] = 0
    return df_result


class AllocationSolver:
    """Solver for optimal share allocation with primary unit constraints"""
    
    def __init__(self, 
                 qty_executed: int,
                 price_executed: float,
                 asset_trade: str,
                 df_funds_target: pd.DataFrame,
                 max_iterations: int = 1000):
        """
        Initialize the allocation solver
        
        Args:
            qty_executed: Total quantity to allocate
            price_executed: Price per share
            asset_trade: Asset identifier
            df_funds_target: DataFrame with fund targets
            max_iterations: Maximum optimization iterations
        """
        self.qty_executed = qty_executed
        self.price_executed = price_executed
        self.asset_trade = asset_trade
        self.df_funds_target = df_funds_target.copy()
        self.max_iterations = max_iterations
        
        # Get primary unit info
        self.primary_units = self._get_primary_units()
        self.lot_amount = self.primary_units["Current Lot Amount"].iloc[0]
        
        # Pre-calculate constants
        self.target_values = self.df_funds_target['value_target'].values
        self.n_funds = len(self.df_funds_target)
        self.max_possible_allocations = np.floor(
            self.target_values / self.price_executed
        ).astype(int)
        
    def _get_primary_units(self) -> pd.DataFrame:
        """Get primary unit information for the asset"""
        return filter_primary_values(self.asset_trade)
    
    def _calculate_metrics(self, allocations: np.ndarray) -> Tuple[float, pd.DataFrame]:
        """Calculate error and metrics for given allocations"""
        df_temp = self.df_funds_target.copy()
        
        # Calculate financial metrics
        df_temp["qty_allocation"] = allocations
        df_temp["financial_allocation"] = allocations * self.price_executed
        df_temp["target_to_primary"] = df_temp["value_target"] - df_temp["financial_allocation"]
        
        # Calculate primary units
        df_temp["qty_primary_units"] = df_temp["target_to_primary"].apply(
            lambda x: max(0, calculate_primary_request(x, self.lot_amount))
        )
        
        # Check for negative primary units
        if (df_temp["target_to_primary"] < 0).any():
            return float('inf'), df_temp
        
        # Calculate final metrics
        df_temp["financial_primary_units"] = df_temp["qty_primary_units"] * self.lot_amount
        df_temp["Squared_error"] = ((df_temp["value_target"] - (df_temp["financial_primary_units"] + df_temp["financial_allocation"]))
              / df_temp["value_target"]) ** 2
        
        return df_temp["Squared_error"].sum(), df_temp
    
    def _is_valid_allocation(self, allocations: np.ndarray) -> bool:
        """Check if allocation satisfies primary unit constraints"""
        financial_allocation = allocations * self.price_executed
        target_to_primary = self.target_values - financial_allocation
        return not (target_to_primary < 0).any()
    
    def _get_initial_allocation(self) -> np.ndarray:
        """Calculate initial allocation respecting constraints"""
        # Initial proportional allocation
        total_target = self.target_values.sum()
        allocations = np.minimum(
            self.max_possible_allocations,
            np.floor(self.target_values / total_target * self.qty_executed).astype(int)
        )
        
        # Distribute remaining shares
        remaining = self.qty_executed - allocations.sum()
        if remaining > 0:
            available = self.max_possible_allocations - allocations
            fractions = np.where(
                available > 0,
                (self.target_values / total_target * self.qty_executed) - allocations,
                0
            )
            
            while remaining > 0 and available.any():
                idx = np.argmax(fractions * (available > 0))
                if available[idx] <= 0:
                    fractions[idx] = 0
                    continue
                    
                test_alloc = allocations.copy()
                test_alloc[idx] += 1
                
                if self._is_valid_allocation(test_alloc):
                    allocations[idx] += 1
                    available[idx] -= 1
                    remaining -= 1
                else:
                    fractions[idx] = 0
                    
        return allocations
    
    def _optimize_allocation(self, initial_allocation: np.ndarray) -> Tuple[np.ndarray, float, pd.DataFrame]:
        """Optimize allocation using local search"""
        best_allocations = initial_allocation.copy()
        best_error, _ = self._calculate_metrics(best_allocations)
        
        shift_sizes = [1, 2, 5, 10]
        
        for _ in range(self.max_iterations):
            improved = False
            
            # Try all shift sizes
            for shift_size in shift_sizes:
                if improved:
                    break
                    
                # Try moving shares between funds
                for i in range(self.n_funds):
                    if improved:
                        break
                        
                    for j in range(self.n_funds):
                        if i == j or best_allocations[i] < shift_size:
                            continue
                            
                        test_allocations = best_allocations.copy()
                        test_allocations[i] -= shift_size
                        test_allocations[j] += shift_size
                        
                        if self._is_valid_allocation(test_allocations):
                            error, _ = self._calculate_metrics(test_allocations)
                            
                            if error < best_error:
                                best_allocations = test_allocations
                                best_error = error
                                improved = True
                                break
            
            if not improved:
                break
        
        _, final_metrics = self._calculate_metrics(best_allocations)
        return best_allocations, best_error, final_metrics
    
    def solve(self) -> pd.DataFrame:
        """
        Solve the allocation problem and return updated DataFrame
        
        Returns:
            DataFrame with optimal allocation and metrics
        """
        # Handle zero inputs
        if self.qty_executed <= 0 or self.price_executed <= 0:
            print("Zero or invalid inputs detected. Returning zero allocation.")
            return create_zero_result(self.df_funds_target, self.lot_amount)
        
        # Get initial allocation
        initial_allocation = self._get_initial_allocation()
        
        # Optimize allocation
        best_allocations, best_error, final_metrics = self._optimize_allocation(initial_allocation)
        
        # Update DataFrame with results
        self.df_funds_target["qty_allocation"] = best_allocations
        self.df_funds_target["qty_allocation_rounded"] = best_allocations
        self.df_funds_target["financial_allocation"] = final_metrics["financial_allocation"]
        self.df_funds_target["target_to_primary"] = final_metrics["target_to_primary"]
        self.df_funds_target["qty_primary_units"] = final_metrics["qty_primary_units"]
        self.df_funds_target["financial_primary_units"] = final_metrics["financial_primary_units"]
        self.df_funds_target["Squared_error"] = final_metrics["Squared_error"]
        
        # Print summary
        print(f"Final squared error: {best_error}")
        print(f"Total shares executed: {best_allocations.sum()}")
        
        return self.df_funds_target

    def solve_with_fixed_primary(self, df_primary_units: pd.DataFrame) -> pd.DataFrame:
        """
        Solve allocation with pre-calculated primary units
        
        Args:
            df_primary_units: DataFrame with columns [fund_name, fund_cnpj, asset, side, primary_units]
            
        Returns:
            DataFrame with optimal allocation and metrics using fixed primary units
        """
        # Handle zero inputs
        if self.qty_executed <= 0 or self.price_executed <= 0:
            print("Zero or invalid inputs detected. Returning zero allocation.")
            return create_zero_result(self.df_funds_target, self.lot_amount)
        
        # Filter primary units for current asset and create CNPJ mapping
        asset_primary = df_primary_units[df_primary_units['asset'] == self.asset_trade].copy()
        
        # Create CNPJ to primary units mapping
        fund_primary_map = {}
        if not asset_primary.empty:
            try:
                asset_primary['primary_units'] = pd.to_numeric(asset_primary['primary_units'], errors='coerce').fillna(0)
                asset_primary['fund_cnpj_clean'] = asset_primary['fund_cnpj'].astype(str).str.strip()
                fund_primary_map = dict(zip(asset_primary['fund_cnpj_clean'], asset_primary['primary_units']))
            except Exception as e:
                print(f"Error creating primary units mapping: {e}")
                fund_primary_map = {}
        
        # Initialize result DataFrame
        df_result = self.df_funds_target.copy()
        
        # Map primary units using CNPJ
        if 'cnpj' in df_result.columns:
            df_result['cnpj_clean'] = df_result['cnpj'].astype(str).str.strip()
            df_result["qty_primary_units"] = df_result['cnpj_clean'].map(fund_primary_map).fillna(0)
        else:
            df_result["qty_primary_units"] = 0
        
        # Ensure numeric types and handle multiplication error
        try:
            df_result["qty_primary_units"] = pd.to_numeric(df_result["qty_primary_units"], errors='coerce').fillna(0)
            lot_amount_numeric = float(self.lot_amount) if self.lot_amount else 0
            df_result["financial_primary_units"] = df_result["qty_primary_units"] * lot_amount_numeric
        except (ValueError, TypeError) as e:
            print(f"Error calculating financial primary units: {e}")
            df_result["qty_primary_units"] = 0
            df_result["financial_primary_units"] = 0
        
        # Calculate remaining target for secondary allocation
        df_result["remaining_target"] = df_result["value_target"] - df_result["financial_primary_units"]
        
        # Check for negative remaining targets
        if (df_result["remaining_target"] < 0).any():
            print("Warning: Some funds have primary allocation exceeding target value")
            df_result["remaining_target"] = df_result["remaining_target"].clip(lower=0)
        
        # Calculate max possible secondary allocations based on remaining targets
        remaining_targets = df_result["remaining_target"].values
        max_secondary_allocations = np.floor(remaining_targets / self.price_executed).astype(int)
        
        # Get initial secondary allocation
        total_remaining = remaining_targets.sum()
        if total_remaining > 0:
            allocations = np.minimum(
                max_secondary_allocations,
                np.floor(remaining_targets / total_remaining * self.qty_executed).astype(int)
            )
            
            # Distribute remaining shares
            remaining = self.qty_executed - allocations.sum()
            if remaining > 0:
                available = max_secondary_allocations - allocations
                fractions = np.where(
                    available > 0,
                    (remaining_targets / total_remaining * self.qty_executed) - allocations,
                    0
                )
                
                while remaining > 0 and available.any():
                    idx = np.argmax(fractions * (available > 0))
                    if available[idx] <= 0:
                        fractions[idx] = 0
                        continue
                        
                    test_alloc = allocations.copy()
                    test_alloc[idx] += 1
                    
                    # Check if allocation doesn't exceed remaining target
                    if test_alloc[idx] * self.price_executed <= remaining_targets[idx]:
                        allocations[idx] += 1
                        available[idx] -= 1
                        remaining -= 1
                    else:
                        fractions[idx] = 0
        else:
            allocations = np.zeros(len(df_result), dtype=int)
        
        # Optimize secondary allocation
        best_allocations = allocations.copy()
        best_error = float('inf')
        
        # Calculate initial error
        financial_allocation = best_allocations * self.price_executed
        total_financial = financial_allocation + df_result["financial_primary_units"].values
        squared_errors = ((df_result["value_target"].values - total_financial) / df_result["value_target"].values) ** 2
        best_error = squared_errors.sum()
        
        # Local search optimization
        shift_sizes = [1, 2, 5, 10]
        
        for _ in range(self.max_iterations):
            improved = False
            
            for shift_size in shift_sizes:
                if improved:
                    break
                    
                for i in range(len(df_result)):
                    if improved:
                        break
                        
                    for j in range(len(df_result)):
                        if i == j or best_allocations[i] < shift_size:
                            continue
                            
                        test_allocations = best_allocations.copy()
                        test_allocations[i] -= shift_size
                        test_allocations[j] += shift_size
                        
                        # Check constraints
                        if (test_allocations[j] * self.price_executed <= remaining_targets[j] and 
                            test_allocations[i] >= 0):
                            
                            # Calculate error
                            test_financial = test_allocations * self.price_executed
                            test_total = test_financial + df_result["financial_primary_units"].values
                            test_errors = ((df_result["value_target"].values - test_total) / df_result["value_target"].values) ** 2
                            test_error = test_errors.sum()
                            
                            if test_error < best_error:
                                best_allocations = test_allocations
                                best_error = test_error
                                improved = True
                                break
            
            if not improved:
                break
        
        # Update result DataFrame with final allocations
        df_result["qty_allocation"] = best_allocations
        df_result["qty_allocation_rounded"] = best_allocations
        df_result["financial_allocation"] = best_allocations * self.price_executed
        df_result["target_to_primary"] = df_result["remaining_target"] - df_result["financial_allocation"]
        
        # Calculate final squared error
        total_financial_final = df_result["financial_allocation"] + df_result["financial_primary_units"]
        df_result["Squared_error"] = ((df_result["value_target"] - total_financial_final) / df_result["value_target"]) ** 2
        
        # Print summary
        print(f"Final squared error: {best_error}")
        print(f"Total shares executed: {best_allocations.sum()}")
        print(f"Total primary units used: {df_result['qty_primary_units'].sum()}")
        
        return df_result

In [31]:
import boto3
import json
import base64
import requests
import pandas as pd
from typing import Dict, Any, Optional, List

class TradingAPIManager:
    def __init__(self, secrets_arn: str, region_name: str = 'us-east-1'):
        """
        Initialize the Trading API Manager
        
        Args:
            secrets_arn (str): ARN for AWS Secrets
            region_name (str): AWS region name
        """
        self.secrets_arn = secrets_arn
        self.region_name = region_name
        self.secrets = self._load_secrets()
        self.inoa_base_url = 'http://10.10.5.8/'  # PROD
        self.posttrades_base_url = 'https://api.postrade.btgpactual.com'

    def _get_secret(self, secret_name: str) -> str:
        """
        Get secret from AWS Secrets Manager
        
        Args:
            secret_name (str): Name of the secret
            
        Returns:
            str: Secret value
        """
        session = boto3.session.Session()
        client = session.client(
            service_name='secretsmanager',
            region_name=self.region_name
        )

        secret_response = client.get_secret_value(SecretId=secret_name)

        if 'SecretString' in secret_response:
            return secret_response['SecretString']
        else:
            return base64.b64decode(secret_response['SecretBinary'])

    def _load_secrets(self) -> Dict[str, str]:
        """
        Load all secrets from AWS
        
        Returns:
            Dict[str, str]: Dictionary of secrets
        """
        secret_str = self._get_secret(self.secrets_arn)
        return json.loads(secret_str)

    def get_post_trades(self, query: str) -> Dict[str, Any]:
        """
        Get data from Post Trades API
        
        Args:
            query (str): API query path
            
        Returns:
            dict: API response
        """
        url = f'{self.posttrades_base_url}{query}'
        headers = {
            'Authorization': f'Bearer {self.secrets["POSTTRADES_API_PASSWORD"]}',
            'Accept': 'application/json',
        }

        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(
                f"Request failed: Code {response.status_code}, "
                f"Message: {response.text}"
            )

    def get_consolidated_trades(self) -> pd.DataFrame:
        """
        Get and process consolidated trades
        
        Returns:
            pd.DataFrame: Processed trades data
        """
        # Get raw trades data
        trades_data = self.get_post_trades('/allocation/consolidated')
        trades_df = pd.DataFrame(trades_data['consolidatedAllocations'])

        # Process trades DataFrame
        trades_df = trades_df[[
            'accountCode', 'accountAlias', 'side', 'symbol', 
            'qty', 'avgPrice'
        ]]
        
        # Rename columns
        trades_df.columns = [
            'CONTA', 'CONTA_NOME', 'SIDE', 'ATIVO', 
            'QTY. AVAILABLE', 'AVG_PRICE'
        ]

        # Transform data
        trades_df['SIDE'] = trades_df['SIDE'].map({'Buy': 'C', 'Sell': 'V'})
        trades_df = trades_df[~trades_df['ATIVO'].str.endswith('H')]
        trades_df['Financeiro'] = trades_df['QTY. AVAILABLE'] * trades_df['AVG_PRICE']

        return trades_df

In [ ]:
from google.cloud import bigquery
from typing import Optional

def truncate_primary_requests_table() -> None:
    """
    Deletes all records from the primary_requests table in BigQuery.
    
    Returns:
        None
    
    Raises:
        Exception: If the deletion operation fails
    """
    try:
        # Define BigQuery parameters
        project_id = "hdx-data-platform-users"
        table_id = "team_trading.primary_requests"
        
        # Initialize BigQuery client
        client = bigquery.Client(project=project_id)
        
        # Create delete query
        delete_query = f"""
        DELETE FROM `{project_id}.{table_id}`
        WHERE TRUE
        """
        
        # Execute query
        query_job = client.query(delete_query)
        query_job.result()  # Wait for query to complete
        
        print(f"Successfully deleted all records from {table_id}")
        
    except Exception as e:
        print(f"Error truncating table: {str(e)}")
        raise
    
    
truncate_primary_requests_table()